# 콘텐츠 기반 추천

In [1]:
import pandas as pd
import numpy as np
import scipy 
import sklearn

In [2]:
path = "anonymous-msweb.test"

In [3]:
path

'anonymous-msweb.test'

In [4]:
import pandas as pd

In [5]:
raw_data = pd.read_csv(path,header=None, skiprows=7)
raw_data.head()

,0,1,2,3,4
0,A,1277,1,NetShow for PowerPoint,/stream
1,A,1253,1,MS Word Development,/worddev
2,A,1109,1,TechNet (World Wide Web Edition),/technet
3,A,1038,1,SiteBuilder Network Membership,/sbnmember
4,A,1205,1,Hardware Supprt,/hardwaresupport


In [6]:
#사용자 활동

In [7]:
user_activity = raw_data.loc[raw_data[0] !="A"] #A가 아닌 레코드만 필터링

In [8]:
user_activity = user_activity.loc[:,:1] #원하지 않는 열 제거

In [9]:
user_activity.columns = ['category','value'] #열 이름 할당

In [10]:
user_activity.head()

,category,value
294,C,10001
295,V,1038
296,V,1026
297,V,1034
298,C,10002


In [11]:
len(user_activity.loc[user_activity['category'] =='V'].value.unique()) #고유 web ID 총합

236

In [12]:
len(user_activity.loc[user_activity['category'] =="C"].value.unique()) #고유 사용자 수 

5000

In [13]:
#사용자-아이템-평가 행렬 생성
tmp = 0 #변수 할당
nextrow = False

In [14]:
lastindex = user_activity.index[len(user_activity)-1] #데이터 세트의 마지막 인덱스 
lastindex

20484

In [15]:
for index,row in user_activity.iterrows():
    if(index <= lastindex ):
        if(user_activity.loc[index,'category'] == "C"):
            tmp = 0
            #add += 1
            #user_activity.loc[index,'chunk'] += add
            userid = user_activity.loc[index,'value']
            user_activity.loc[index,'userid'] = userid
            user_activity.loc[index,'webid'] = userid
            tmp = userid
            nextrow = True            
        elif(user_activity.loc[index,'category'] != "C" and nextrow == True):
                #user_activity.loc[index,'chunk'] += add
                webid = user_activity.loc[index,'value']
                user_activity.loc[index,'webid'] = webid
                user_activity.loc[index,'userid'] = tmp
                if(index != lastindex and user_activity.loc[index+1,'category'] == "C"):
                    nextrow = False
                    caseid = 0

In [16]:
user_activity.head()

,category,value,userid,webid
294,C,10001,10001.0,10001.0
295,V,1038,10001.0,1038.0
296,V,1026,10001.0,1026.0
297,V,1034,10001.0,1034.0
298,C,10002,10002.0,10002.0


In [17]:
user_activity = user_activity[user_activity['category'] == "V"] #원치 않는 행 제거

In [18]:
user_activity.head()

,category,value,userid,webid
295,V,1038,10001.0,1038.0
296,V,1026,10001.0,1026.0
297,V,1034,10001.0,1034.0
299,V,1008,10002.0,1008.0
300,V,1056,10002.0,1056.0


In [19]:
user_activity = user_activity[['userid','webid']]

In [20]:
user_activity.head()

,userid,webid
295,10001.0,1038.0
296,10001.0,1026.0
297,10001.0,1034.0
299,10002.0,1008.0
300,10002.0,1056.0


In [21]:
user_activity_sort = user_activity.sort_values('webid',ascending=True) #데이터를 webid 정렬 ->DF에서는 sort_values 함수 사용

In [22]:
#user_item_rating을 포함하는 조밀한 이진 평가 행렬 생성
sLength = len(user_activity_sort['webid']) #webid 열의 크기 확인

In [23]:
sLength

15191

In [24]:
import numpy as np

In [25]:
user_activity_sort['rating'] = pd.Series(np.ones((sLength,)), index=user_activity.index) #1만 있는 rating 열 추가

In [26]:
ratmat = user_activity_sort.pivot(index='userid', columns='webid', values='rating').fillna(0)

In [27]:
ratmat

webid,1000.0,1001.0,1002.0,1003.0,1004.0,1005.0,1006.0,1007.0,1008.0,1009.0,...,1252.0,1253.0,1256.0,1257.0,1258.0,1261.0,1266.0,1278.0,1295.0,1297.0
userid,,,,,,,,,,,,,,,,,,,,,
10001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14996.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
ratmat = ratmat.todense()

AttributeError: 'DataFrame' object has no attribute 'todense'

# 아이템 프로필 생성

In [40]:
items = raw_data.loc[raw_data[0] =="A"]

In [41]:
items.columns = ['record','webid','vote','desc','url']

In [42]:
items = items[['webid','desc']]

In [43]:
items.head()

,webid,desc
0,1277,NetShow for PowerPoint
1,1253,MS Word Development
2,1109,TechNet (World Wide Web Edition)
3,1038,SiteBuilder Network Membership
4,1205,Hardware Supprt


In [44]:
items['webid'].unique().shape[0]

294

In [45]:
#user_activity 데이터의 webid 만 확인
items2 = items[items['webid'].isin(user_activity['webid'].tolist())]

In [48]:
#webid로 데이터 정렬
items_sort = items2.sort_values('webid', ascending=True)

In [49]:
items_sort

,webid,desc
113,1000,regwiz
40,1001,Support Desktop
278,1002,End User Produced View
102,1003,Knowledge Base
243,1004,Microsoft.com Search
...,...,...
70,1261,MS's Complete Do It Yourself Guide
33,1266,Licenses and Piracy
30,1278,MS in Higer Education
115,1295,Training


In [52]:
#TF-IDF 아이템 프로필 생성
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
v = TfidfVectorizer(stop_words = "english", max_features = 100, ngram_range = (0,3),sublinear_tf = True)

In [54]:
x = v.fit_transform(items_sort['desc'])

In [57]:
x

<236x100 sparse matrix of type '<class 'numpy.float64'>'
	with 619 stored elements in Compressed Sparse Row format>

In [55]:
itemprof = x.todense()

# 사용자 프로필 생성

In [59]:
from scipy import linalg, dot

In [61]:
userprof = dot(ratmat,itemprof)/linalg.norm(ratmat)/linalg.norm(itemprof) #사용자활동과 아이템프로필의 내적 계산

C:\Users\KIM\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead
  """Entry point for launching an IPython kernel.


In [62]:
userprof

matrix([[0.00062937, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00089668, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00144708, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.00046412, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00067229, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00079067, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [63]:
#아이템에 대한 활성 사용자의 선호도를 계산:사용자프로필-아이템프로필 코사인 유사도 사용
import sklearn.metrics
similarityCalc = sklearn.metrics.pairwise.cosine_similarity(userprof, itemprof, dense_output=True)

In [64]:
similarityCalc

array([[0.54168902, 0.17449812, 0.23677035, ..., 0.20670579, 0.16290362,
        0.19718935],
       [0.78844617, 0.25398775, 0.34462703, ..., 0.30086706, 0.23711158,
        0.28701558],
       [0.63172381, 0.20350167, 0.27612424, ..., 0.29413451, 0.18998003,
        0.22996444],
       ...,
       [0.56969503, 0.1835199 , 0.24901168, ..., 0.21739274, 0.17132595,
        0.20738429],
       [0.49394733, 0.15911875, 0.21590263, ..., 0.1884878 , 0.14854613,
        0.17981009],
       [0.86518334, 0.27870764, 0.37816858, ..., 0.33014958, 0.26018896,
        0.31494998]])

In [66]:
final_pred = np.where(similarityCalc > 0.6, 1, 0) #이진데이터로 전환

In [68]:
indexs_of_user = np.where(final_pred[213] == 1 )